In [1]:
#http://127.0.0.1:8888/?token=641ff58c730676d8f40d01ea1af56b3dc8e8be248d9aeb4a
import os
import re
import operator
import matplotlib.pyplot as plt
import warnings
import gensim
import numpy as np
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from pprint import pprint
import json
import pyLDAvis.gensim
import datetime
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import *
import numpy as np
import string
%matplotlib inline
import pandas as pd

In [38]:
# import file to calculate LDA
# REDDIT
filename = 'bitcoin_2019_august_data.csv'
df = pd.read_csv(os.path.join(os.getcwd(), 'reddit_data', 'clean_data', filename))

# TELEGRAM
# filename = '_Telegram_cleanedData_2019-09-01_2019-09-17.csv'
# df = pd.read_csv(os.path.join(os.getcwd(), 'telegram_data', 'clean_data', filename))
# df = df.iloc[:,1:]
# print(df.columns)

In [39]:
df.columns

Index(['Unnamed: 0', 'subreddit', 'body', 'created_utc', 'author'], dtype='object')

In [40]:
# Select only data desidered:
# df = df[df.author == 'mindy2000']
df1 = df[df.subreddit == 'Futurology']
print(len(df))

60715


In [41]:
def build_texts(fname):
    """
    Function to build tokenized texts from file
    fname: File to be read
    Returns: yields preprocessed line
    """
    
    for line in fname:
        yield gensim.utils.simple_preprocess(line, deacc=True, min_len=3)
            

def clean_text(text):
    try:
        stop_words = set(stopwords.words('english'))
        #porter = PorterStemmer()
        #lemman = LancasterStemmer()
        translator = str.maketrans('', '', string.punctuation)
        text = re.sub(r'http\S+', '', text)
        text = text.translate(translator)
        return ' '.join([w for w in text.split() if w not in stop_words])
        #return ' '.join([lemman.stem(porter.stem(w)) for w in text.split() if w not in stop_words])

    except:
        return ''


In [42]:
# Clean text in dataframe
cleaned_data = [clean_text(d) for d in df1['body'].tolist()]
train_texts = list(build_texts(cleaned_data))
print(f'Number of messages --> {len(train_texts)}')

Number of messages --> 101


In [43]:
dictionary = Dictionary(train_texts)
corpus = [dictionary.doc2bow(text) for text in train_texts]

In [44]:
import random
random.seed(113)
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)

In [45]:
pyLDAvis.enable_notebook()

In [46]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.047847 -0.003964       1        1  23.184690
0     -0.007158  0.015627       2        1  22.080412
2     -0.011015 -0.001791       3        1  19.583744
3      0.036065  0.032429       4        1  17.636490
1      0.029955 -0.042300       5        1  17.514668, topic_info=     Category       Freq             Term      Total  loglift  logprob
term                                                                  
779   Default  16.000000            libra  16.000000  30.0000  30.0000
1063  Default   4.000000           please   4.000000  29.0000  29.0000
73    Default  42.000000            would  42.000000  28.0000  28.0000
262   Default  17.000000             dont  17.000000  27.0000  27.0000
1060  Default   3.000000       moderators   3.000000  26.0000  26.0000
1071  Default   3.000000          removal   3.000000  25.0000  25.0000
1074  Default   3.000000       submission   3.000000  24.0000  24.0000
1070  Default   3.000000        regarding   3.000000  23.0000  23.0000
13    Default  10.000000             good  10.000000  22.0000  22.0000
709   Default   5.000000         identity   5.000000  21.0000  21.0000
527   Default   3.000000              bch   3.000000  20.0000  20.0000
250   Default  10.000000              but  10.000000  19.0000  19.0000
3     Default   3.000000             meat   3.000000  18.0000  18.0000
2007  Default   2.000000      millennials   2.000000  17.0000  17.0000
8     Default   4.000000              buy   4.000000  16.0000  16.0000
212   Default  20.000000            money  20.000000  15.0000  15.0000
353   Default   5.000000            check   5.000000  14.0000  14.0000
351   Default   5.000000             year   5.000000  13.0000  13.0000
1335  Default   6.000000           client   6.000000  12.0000  12.0000
1     Default  96.000000          bitcoin  96.000000  11.0000  11.0000
1987  Default   2.000000       uoliaustin   2.000000  10.0000  10.0000
1098  Default   2.000000       experiment   2.000000   9.0000   9.0000
334   Default  12.000000           system  12.000000   8.0000   8.0000
1059  Default   4.000000          message   4.000000   7.0000   7.0000
504   Default   4.000000             gold   4.000000   6.0000   6.0000
1115  Default   3.000000         becoming   3.000000   5.0000   5.0000
443   Default   3.000000           stolen   3.000000   4.0000   4.0000
151   Default   3.000000              per   3.000000   3.0000   3.0000
1079  Default   2.000000          twitter   2.000000   2.0000   2.0000
1051  Default   2.000000         concerns   2.000000   1.0000   1.0000
...       ...        ...              ...        ...      ...      ...
149    Topic5   3.042267              its  11.390487   0.4220  -5.6593
545    Topic5   1.708675           happen   4.289451   0.8217  -6.2361
12     Topic5   4.148722              use  19.870325   0.1757  -5.3491
200    Topic5   4.649075             like  26.220272   0.0123  -5.2352
431    Topic5   1.520909          protect   3.621799   0.8745  -6.3525
1366   Topic5   1.149042  implementations   2.205728   1.0900  -6.6329
960    Topic5   2.568415      blockchains   9.348014   0.4503  -5.8286
2048   Topic5   1.140348            trump   2.178862   1.0947  -6.6405
363    Topic5   5.440928       blockchain  39.366501  -0.2368  -5.0779
420    Topic5   1.629339              man   4.344012   0.7615  -6.2837
493    Topic5   3.486719              you  18.436802   0.0767  -5.5229
198    Topic5   2.511638             know   9.975674   0.3629  -5.8509
589    Topic5   1.469255             core   3.665009   0.8281  -6.3871
274    Topic5   1.720440             free   5.076236   0.6601  -6.2293
221    Topic5   2.720619             real  12.711017   0.2005  -5.7710
457    Topic5   3.248302             vote  18.438978   0.0058  -5.5937
301    Topic5   2.911005           people  15.087572   0.0968  -5.7034
387    Topic5   2.027919     